In [ ]:
from pathlib import Path
import sys

from dotenv import load_dotenv


sys.path.append(str(Path.cwd().parent))

from libraries.client_stashapp import StashAppClient, get_stashapp_client


load_dotenv()

stash_client = StashAppClient()
stash_raw_client = get_stashapp_client()

In [ ]:
# %%
stashapp_studio_id = 1190
scene_id = 31157
ph_scene = stash_client.find_scenes({"id": {"value": scene_id, "modifier": "EQUALS"}})
ph_scene

import json


primary_file_path = ph_scene.to_dicts()[0]["stashapp_primary_file_path"]

# Check if .info.json file exists. Original suffix needs to be removed.
primary_file_path_obj = Path(primary_file_path)
info_json_path = str(primary_file_path_obj.with_suffix("")) + ".info.json"

print(info_json_path)

if Path(info_json_path).exists():
    with open(info_json_path, "r") as f:
        info_json = json.load(f)
        # pretty print
        print(json.dumps(info_json, indent=4))

import base64
from datetime import datetime

import requests


thumbnail_url = info_json["thumbnail"]

# Download thumbnail and convert to base64
thumbnail_response = requests.get(thumbnail_url)
thumbnail_base64 = base64.b64encode(thumbnail_response.content).decode("utf-8")
cover_image_base64 = f"data:image/jpeg;base64,{thumbnail_base64}"


stash_raw_client.update_scene(
    {
        "id": ph_scene.to_dicts()[0]["stashapp_id"],
        "code": info_json["id"],
        "title": info_json["title"],
        "date": datetime.strptime(info_json["upload_date"], "%Y%m%d").strftime(
            "%Y-%m-%d"
        ),
        "urls": info_json["webpage_url"],
        "cover_image": cover_image_base64,
        "studio_id": stashapp_studio_id,
    }
)